# Modelling 

In [5]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium 
from pyspark.ml.feature import VectorAssembler, StandardScaler, OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


In [6]:
# Create a spark session with increased memory allocation
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "8g")  # Set the driver memory to 8GB
    .config("spark.executor.memory", "8g")  # Set the executor memory to 8GB
    .getOrCreate()
)

### Dataset

In [17]:
# import the data
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [18]:
# Load the data
df = spark.read.parquet("../data/curated/tlc_data/first_cleaned.parquet")

### Standardization

### Categorical Encoding

In [ ]:
# Indexing the categorical columns
vendor_indexer = StringIndexer(inputCol="VendorID", outputCol="VendorID_index")
ratecode_indexer = StringIndexer(inputCol="RatecodeID", outputCol="RatecodeID_index")
payment_type_indexer = StringIndexer(inputCol="payment_type", outputCol="payment_type_index")

# OneHotEncoding the indexed columns
vendor_encoder = OneHotEncoder(inputCol="VendorID_index", outputCol="VendorID_vec")
ratecode_encoder = OneHotEncoder(inputCol="RatecodeID_index", outputCol="RatecodeID_vec")
payment_type_encoder = OneHotEncoder(inputCol="payment_type_index", outputCol="payment_type_vec")

# Creating a pipeline to chain indexers and encoders
pipeline = Pipeline(stages=[vendor_indexer, ratecode_indexer, payment_type_indexer,
                            vendor_encoder, ratecode_encoder, payment_type_encoder])

# Fit the pipeline and transform the data
model = pipeline.fit(combined)
encoded_df = model.transform(combined)

# Drop the original columns after encoding
encoded_df = encoded_df.drop("VendorID", "RatecodeID", "payment_type")

## Data split

## Linear regression

In [ ]:
# Assembling the feature vector
assembler = VectorAssembler(inputCols=[<your_selected_columns>], outputCol="features")

# Standardization
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Linear Regression
lr = LinearRegression(featuresCol="scaled_features", labelCol="hourly_trip_count")

# Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler, lr])

# Fit the model
model = pipeline.fit(train_df)

# Evaluate the model
predictions = model.transform(test_df)
evaluator = RegressionEvaluator(labelCol="hourly_trip_count", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

print(f"Root Mean Squared Error (RMSE) on test data = {rmse}") 

## Random Forest 